# Part 2: Indexing and Evaluation

In [24]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import math
from collections import defaultdict, Counter

import string
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [4]:
# Data import
data_path =  os.path.join(os.getcwd(), '../../data/')
doc_path = os.path.join(data_path, 'fashion_products_dataset.json')

data = pd.read_csv(os.path.join(data_path, 'fashion_products_cleaned.csv'))

data[2640:2660]

,pid,title,description,brand,category,sub_category,seller,out_of_stock,selling_price,discount,...,detail_size,detail_neck_type,detail_country_of_origin,detail_brand_fit,detail_sleeve_type,detail_other_details,detail_model_name,detail_occasion,detail_closure,detail_secondary_color
2640,CTPECRYPEUBGYFBE,tripin brass cufflink beig,tripin bieg color cufflink amaz whirl glitter ...,NaN,clothing and accessories,clothing accessories,NaN,1,300.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,tsil41,NaN,NaN,NaN
2641,CTPEA93EZXHZWD4C,tripin brass cufflink silver,beauti item brand new come complimentari gift ...,NaN,clothing and accessories,clothing accessories,tripin,0,349.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,black and white royal design rectangle cufflinks,NaN,NaN,NaN
2642,CTPE68FG4RNYYQYV,tripin brass cufflink set silver black,tripin brass cufflink complet attir,NaN,clothing and accessories,clothing accessories,NaN,1,500.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,classic,NaN,NaN,NaN
2643,CTPEHCEPKHHACGAA,tripin brass cufflink tie pin set multicolor,cufflink set man apart feel better add cufflin...,NaN,clothing and accessories,clothing accessories,NaN,1,499.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,tripin cufflinks with matching tie pin,NaN,NaN,NaN
2644,CTPFDY3GXVEB6AUU,tripin brass cufflink silver,NaN,NaN,clothing and accessories,clothing accessories,tripin,0,499.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,silver with mirror,NaN,NaN,NaN
2645,CTPEA93EYCWCDH7M,tripin brass cufflink gold,beauti item brand new come complimentari gift ...,NaN,clothing and accessories,clothing accessories,NaN,1,500.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,golden unique shaped cufflinks with stone with...,NaN,NaN,NaN
2646,CTPEM2CY3SUR8PDX,tripin brass cufflink silver,cufflink set man apart feel better add cufflin...,NaN,clothing and accessories,clothing accessories,NaN,1,299.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,tripin silver cufflinks for men in a gift box,NaN,NaN,NaN
2647,CTPEG6GQHUKEM8EW,tripin brass cufflink blue,cufflink set woman apart feel better add cuffl...,NaN,clothing and accessories,clothing accessories,NaN,1,499.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,tripin golden square blue diamond crystals cuf...,NaN,NaN,NaN
2648,CTPEFTS5RAYWMNQX,tripin brass cufflink tie pin set silver,cufflink set man apart feel better add cufflin...,NaN,clothing and accessories,clothing accessories,NaN,1,199.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,silver square cufflink gift set with amazing s...,NaN,NaN,NaN
2649,CTPEA9ZFNTYSP4DY,tripin brass cufflink gold,beauti item brand new come complimentari gift ...,NaN,clothing and accessories,clothing accessories,NaN,1,300.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,golden round cufflinks with brown stone with a...,NaN,NaN,NaN


## 1. Indexing

### 1.1 Build inverted index

After having pre-processed the data, you can then create the inverted index.

HINT - you may use the vocabulary data structure, like the one seen during the
Practical Labs:

{
    Term_id_1: [document_1, document_2, document_4],
    Term_id_2: [document_1, document_3, document_5, document_6],
    etc…
}

Important: For this assignment, we will be using conjunctive queries (AND). This means that every returned document must contain all the words from the query in order to be considered a match.

In [46]:
# Preprocessing function used in PART-1
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))
translator = str.maketrans('', '', string.punctuation)

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower() # Lowercase
    text = text.translate(translator) # Remove punctuation
    text = unidecode(text) # normalize
    tokens = word_tokenize(text) # Tokenization
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words] # Remove stopwords and non-alphabetic tokens
    stemmed_tokens = [stemmer.stem(word) for word in tokens] # Stemming 
    stemmed_tokens = [word for word in stemmed_tokens if len(word) > 2] # Remove short tokens
    return ' '.join(stemmed_tokens)


In [ ]:
# 1. Combining text fields from the dataset to create a searchable document for each product.
def create_document_for_each_row(row, text_columns):
    """ 
    Create a single text document for each row by concatenating specified text fields
    Returns: string of concatenated text fields
    """
    text_fields = [str(row[col]) for col in text_columns if pd.notnull(row[col])]
    document = ' '.join([f for f in text_fields if f != 'nan' and f != '']).lower()
    return document

# 2. Building an inverted index to map terms to product IDs.
def build_inverted_index(data, text_columns):
    """
    Build an inverted index from the dataset
    Returns: dict mapping terms to list of document IDs
    """
    inverted_index = defaultdict(list)

    for _, row in data.iterrows():
        doc_id = row['pid']
        row_text = create_document_for_each_row(row, text_columns) 

        ####### SI?
        row_text = preprocess_text(row_text)

        # Tokenize the text
        tokens = re.findall(r'\b\w+\b', row_text)

        # Add to inverted index (avoindin duplicates)
        already_seen_terms = set()
        for term in tokens:
            if term not in already_seen_terms:
                inverted_index[term].append(doc_id)
                already_seen_terms.add(term)

    return dict(inverted_index)

# 3. Implementing a simple search function to retrieve products based on keyword queries. (conjunctive queries)
def conjunctive_search(query, inverted_index):
    """
    Perform AND query: return documents containing ALL QUERY TERMS
    Returns: list of document IDs
    """
    query_terms = re.findall(r'\b\w+\b', query.lower())
    if not query_terms:
        return []

    # Get documents for the first term
    if query_terms[0] in inverted_index:
        result_docs = set(inverted_index[query_terms[0]])
    else:
        return []

    # Intersect with documents for the remaining terms
    for term in query_terms[1:]:
        if term in inverted_index:
            result_docs = result_docs.intersection(set(inverted_index[term]))
        else:
            return []

    return list(result_docs)


In [48]:
text_columns = ['title', 'description', 'brand', 'category', 'sub_category', 'seller']
# TODO: detail_columns = ['detail_fabric', 'detail_color', 'detail_pattern', ...]
# For now we are building it without the details columns

# --- Our inverted index ---
inverted_index = build_inverted_index(data, text_columns)

### 1.2 Propose test queries

Define five queries that will be used to evaluate your search engine. (Be creative 😉)

HINT: How to choose the queries? The selection of the queries is up to you, but it’s suggested to select terms based on the popularity (keywords ranked by term frequencies or by TF-IDF, etc).

In [49]:
test_queries = [
    "men cotton shirt",
    "women casual polo neck",
    "men regular fit tshirt",
    "zipper sweater",
    "solid round neck cotton"
]

for query in test_queries:
    results = conjunctive_search(query, inverted_index)
    print(f"\nTest query: '{query}'")
    print(f"Found {len(results)} matching documents")
    print(f"Sample results: {results[:5]}")


Test query: 'men cotton shirt'
Found 910 matching documents
Sample results: ['TSHFNQWHNNFZZCRF', 'TSHFTT25AZSPCFKX', 'TSHFSFVQTJTYFN2E', 'SHTFK89ZMYAUHGG3', 'TSHFGHVFGBCFXHFH']

Test query: 'women casual polo neck'
Found 181 matching documents
Sample results: ['TSHFUBHZFZA6GE9P', 'TSHFT3NCX83HC8CW', 'TSHFY6ZWVSUZVE22', 'TSHFKJGYBYHPHMGG', 'TSHES4E2GBEZEAPG']

Test query: 'men regular fit tshirt'
Found 286 matching documents
Sample results: ['TSHFNQWHNNFZZCRF', 'SHTFW5F25PKVGCCG', 'SHTFWTZV8HJRDAT2', 'TSHFNV9K2RBPGQH4', 'JEAFGUGYBV6KFMNV']

Test query: 'zipper sweater'
Found 1 matching documents
Sample results: ['RNCF4YV4GZRYJY3A']

Test query: 'solid round neck cotton'
Found 515 matching documents
Sample results: ['TSHFXYERRWUGYDZG', 'TSHFVGBUY6DEZGEZ', 'TSHFFEZ2XHH4T2AH', 'TSHEW38EYJSYSMJQ', 'TSHEM8ZMMUXPJCQW']


### 1.3 Rank your results

Implement the TF-IDF algorithm and provide ranking-based results.

In [50]:
# N = number of documents
# dft = number of documents containing term t

# 1. Create the TF-IDF index
def create_index_tfidf(data, text_columns):
    """
    Build TF-IDF index from the dataset
    Returns: 
        - tf_index: dict mapping (doc_id, term) -> term frequency
        - idf_index: dict mapping term -> inverse document frequency
        - doc_lengths: dict mapping doc_id -> total terms in document
    """
    N = len(data)
    tf_index = defaultdict(list) # (doc_id, term) -> count
    df_index = defaultdict(int) # term -> num of docs containing term
    idf_index = defaultdict(list) # term -> idf

    doc_lenghts = {} # doc_id -> total number of terms in doc

    for idx, row in data.iterrows():
        # First steps as before
        doc_id = row["pid"]
        row_text = create_document_for_each_row(row, text_columns)
        tokens = re.findall(r'\b\w+\b', row_text) # Tokenize

        # Calculate term freqs for this doc
        term_counts = Counter(tokens)
        doc_lenghts[doc_id] = len(tokens)

        # Store TF and update DF
        for term, count in term_counts.items():
            tf_index[(doc_id, term)] = count
            df_index[term] += 1

    # Calculate IDF for each term
    for term, df in df_index.items():
        idf_index[term] = math.log(N / df)

    return tf_index, idf_index, doc_lenghts


# 2. Calculate TF-IDF scoring for doc-query pairs
def calculate_score_tfidf(doc_id, query_terms, tf_index, idf_index, doc_lengths):
    """ 
    Calculate TF-IDF score for a document given the query terms
    Returns: TF-IDF score (float)
    """
    score = 0.0

    for term in query_terms:
        tf = tf_index.get((doc_id, term), 0)

        if tf > 0:
            # Normalized TF
            norm_tf = tf / doc_lengths[doc_id]
            # IDF for the term
            idf = idf_index.get(term, 0)
            # TF-IDF score
            score += norm_tf * idf

    return score


# 3. Ranked serach with TF-IDF
def ranked_search_tfidf(query, inverted_index, tf_index, idf_index, doc_lengths, top_k=10):
    """
    Perform conjunctive search and rank results by TF-IDF score
    Returns: list of (doc_id, score) tuples sorted by score descending
    """
    # Get documents matching all query terms
    documents = conjunctive_search(query, inverted_index)
    if not documents:
        return []
    
    # Tokenize query
    query_terms = re.findall(r'\b\w+\b', query.lower())

    # Caluclate TF-IDF scores for each document
    doc_scores = []
    for doc_id in documents:
        score = calculate_score_tfidf(doc_id, query_terms, tf_index, idf_index, doc_lengths)
        doc_scores.append((doc_id, score))

    # Sort documents by score descending
    ranked_docs = sorted(doc_scores, key=lambda x: x[1], reverse=True)

    # Return top K results
    return ranked_docs[:top_k]


In [51]:
# --- Our TF-IDF index ---
tf_index, idf_index, doc_lengths = create_index_tfidf(data, text_columns)
queries = test_queries

# Prints done with copilot
for query in queries:
    ranked_results = ranked_search_tfidf(query, inverted_index, tf_index, idf_index, doc_lengths, top_k=10)
    
    print(f"\nTest query: '{query}'")
    print(f"Found {len(ranked_results)} ranked documents")
    
    if ranked_results:
        print(f"Top 5 results:")
        for rank, (doc_id, score) in enumerate(ranked_results[:5], 1):
            # Retrieve product information for display
            product = data[data['pid'] == doc_id].iloc[0]
            title = product['title']
            if len(str(title)) > 50:
                title = str(title)[:50] + "..."
            
            print(f"  {rank}. PID: {doc_id} | Score: {score:.4f}")
            print(f"     Title: {title}")
    else:
        print("  No results found")
    print("-" * 70)


Test query: 'men cotton shirt'
Found 10 ranked documents
Top 5 results:
  1. PID: SHTFEBYPPJQ5SN48 | Score: 0.4469
     Title: men slim fit checker casual shirt
  2. PID: SHTFEBYPME8WWM34 | Score: 0.4125
     Title: men slim fit print spread collar casual shirt
  3. PID: SHTFEBYPVFWSSEEA | Score: 0.3972
     Title: men slim fit color block spread collar casual shir...
  4. PID: SHTFZQJFTJ6YCH2P | Score: 0.3806
     Title: men slim fit print casual shirt
  5. PID: SHTFZQJFGZYKGSNZ | Score: 0.3806
     Title: men slim fit print casual shirt
----------------------------------------------------------------------

Test query: 'women casual polo neck'
Found 10 ranked documents
Top 5 results:
  1. PID: TSHFUMM2UQCQV5QA | Score: 0.3773
     Title: stripe women polo neck pink tshirt
  2. PID: TSHFUMM2WXY73YHC | Score: 0.3622
     Title: stripe women polo neck dark blue tshirt
  3. PID: TSHFUMM2F6VHKRDJ | Score: 0.3622
     Title: stripe women polo neck light blue tshirt
  4. PID: TSHFTN8K9JUSG

## 2. Evaluation

### 2.1 Implement the following evaluation metrics to assess the effectiveness of your retrieval solutions. 

These metrics will help you measure how well your system retrieves relevant documents for each query:

i. Precision@K (P@K)

ii. Recall@K (R@K)

iii. Average Precision@K (P@K)

iv. F1-Score@K

v. Mean Average Precision (MAP)

vi. Mean Reciprocal Rank (MRR)

vii. Normalized Discounted Cumulative Gain (NDCG)

In [64]:
# We will define functions for each evaluation metric.

# Precision is the share of retrieved documents that are relevant.
def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    relevant_retrieved = [doc for doc in retrieved_k if doc in relevant]
    return len(relevant_retrieved) / k if k > 0 else 0.0

# Recall is the share of relevant documents that are retrieved.
def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    relevant_retrieved = [doc for doc in retrieved_k if doc in relevant]
    return len(relevant_retrieved) / len(relevant) if len(relevant) > 0 else 0.0

# Average Precision is the average of precision scores at each rank position where a relevant document is found
def average_precision_at_k(retrieved, relevant, k): 
    score = 0.0
    num_hits = 0.0
    for i, doc_id in enumerate(retrieved[:k]):
        if doc_id in relevant:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / len(relevant) if len(relevant) > 0 else 0

# F1-score is the harmonic mean of precision and recall.
def f1_score_at_k(retrieved, relevant, k):
    prec = precision_at_k(retrieved, relevant, k)
    rec = recall_at_k(retrieved, relevant, k)
    if prec + rec == 0:
        return 0.0
    return 2 * (prec * rec) / (prec + rec)

# Mean Average Precision (MAP) is the mean of average precision scores across multiple queries.
def mean_average_precision(queries_retrieved, queries_relevant, k):
    ap_scores = []
    for query in queries_retrieved:
        retrieved = queries_retrieved[query]
        relevant = queries_relevant.get(query, [])
        ap = average_precision_at_k(retrieved, relevant, k)
        ap_scores.append(ap)
    return np.mean(ap_scores) if ap_scores else 0.0

# Mean Reciprocal Rank (MRR) is the average of the reciprocal ranks of the first relevant document across multiple queries.
def mean_reciprocal_rank(queries_retrieved, queries_relevant):
    rr_scores = []
    for query in queries_retrieved:
        retrieved = queries_retrieved[query]
        relevant = queries_relevant.get(query, [])
        rank = 0
        for i, doc_id in enumerate(retrieved):
            if doc_id in relevant:
                rank = i + 1
                break
        if rank > 0:
            rr_scores.append(1.0 / rank)
        else:
            rr_scores.append(0.0)
    return np.mean(rr_scores) if rr_scores else 0.0

# Normalized Discounted Cumulative Gain (NDCG) measures the graded relevance of the retrieved documents.
def ndcg_at_k(retrieved, relevant, k):
    relevance_scores = [1 if doc in relevant else 0 for doc in retrieved[:k]]
    ideal_list = sorted(relevance_scores, reverse=True)
    dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores))
    ideal_dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(ideal_list))
    return float(dcg / ideal_dcg) if ideal_dcg > 0 else 0.

### 2.2 Apply Evaluation Metrics
Apply the evaluation metrics you have implemented to the search results and relevance judgments provided in validation_labels.csv for the predefined queries. When reporting evaluation results, provide only numeric values, rounded to three decimal places. Do not include textual explanations or additional statistics in this section.

a. Query 1: women full sleeve sweatshirt cotton

b. Query 2: men slim jeans blue

In [53]:
val_data = pd.read_csv(os.path.join(data_path, 'validation_labels.csv'))
display(val_data.head())

,title,pid,query_id,labels
0,Full Sleeve Printed Women Sweatshirt,SWSFFVKBCQG5FHPF,1,1
1,Full Sleeve Striped Women Sweatshirt,SWSFJY5ZFHQ7HXKW,1,0
2,Full Sleeve Printed Women Sweatshirt,SWSFUY89NHMZHZPX,1,1
3,Full Sleeve Graphic Print Women Sweatshirt,SWSFXQ5YX6RZKHP4,1,1
4,Full Sleeve Solid Women Sweatshirt,JCKFTZBC3DMCVYXH,1,0


In [54]:
# We need to preprocess the query the same way we did the for full data
queries = {
    1: "women full sleeve sweatshirt cotton",
    2: "men slim jeans blue"
}

# Preprocess the query texts
for qid in queries:
    queries[qid] = preprocess_text(queries[qid])

In [55]:
# List of relevant product IDs
queries_relevant = {}
for _, row in val_data.iterrows():
    qid = row['query_id']
    pid = row['pid']
    label = row['labels']
    if qid not in queries_relevant:
        queries_relevant[qid] = []
    if label == 1:
        queries_relevant[qid].append(pid)
print(queries_relevant)

# Retrieve and rank documents for each query
queries_retrieved = {}
for qid, query_text in queries.items():
    ranked_results = ranked_search_tfidf(query_text, inverted_index, tf_index, idf_index, doc_lengths, top_k=20)
    queries_retrieved[qid] = [doc_id for doc_id, _ in ranked_results]  # key = query ID
print(queries_retrieved)

{1: ['SWSFFVKBCQG5FHPF', 'SWSFUY89NHMZHZPX', 'SWSFXQ5YX6RZKHP4', 'SWSFWCTDHRABJFCR', 'SWSFVGKENHBSVUWJ', 'JCKF7M8DNBB6WZ8Y', 'SWSFWCSPEGMDH8FV', 'SWSFXYRYTHZWSZPE', 'SWSFW6BQ74JCVHHH', 'SWSF5R7F38FNMWXG', 'SWSFWEFYEW4AZYTZ', 'SWSFN8YZXFSUCAJX', 'SWSFMJGS8HVBPEH6'], 2: ['JEAFVXG4GGZH9VFA', 'JEAF8CHSVCP5GUH9', 'JEAFUZXRDZWSYWGG', 'JEAFTGSGTYKZGAEZ', 'JEAFUZXRDYSVFK4Y', 'JEAE32FSQ4JXYJK6', 'JEAFUPSWVCTKXMHF', 'JEAFSKYHRVZSABPR', 'JEAFUZXSQQHZXRYM', 'JEAFUZXSMRFFNGC2']}
{1: ['SWSFWTNDJFCF72WU', 'SWSFWTND3EPMFCQD', 'SWSFMJF98EY2FXBH', 'SWSFW6KB8GZHEC72', 'SWSF7NWDHECBTVTF', 'SWSFVEV2PUBUWDDZ', 'SWSFVEV2DRCUFYQK', 'SWSFVEV2KG5K7VWF', 'SWSFVEV2EC45HCYH', 'SWSFVEV2S4YNGS55', 'SWSFV5JN5TKJWPZ2', 'SWSFP4V9B6MGRMFB', 'SWSFNMS5FFRGFQTK', 'SWSFNMS5N4TEXFSG', 'SWSF5R7F2ZYCZYKH', 'SWSFW6BQ74JCVHHH', 'SWSFW7F6GRN8XWCH', 'SWSFFVKBAZGKCQCX', 'SWSF7NWDXMGS3VZV', 'SWSFN8YZZAN8RBDC'], 2: ['JEAF2QF6HNZHDXVX', 'JEAFRAQXEKGUPNUN', 'JEAFSGSYEAFCYHEE', 'JEAF8HYN5B9ZU6FP', 'JEAFTGSGTYKZGAEZ', 'JEAF65G3GPYHV7XW',

In [65]:
k = 20

results = {}

for qid in queries:
    retrieved = queries_retrieved[qid]
    relevant = queries_relevant.get(qid, [])

    results[qid] = {
        'Precision@K': precision_at_k(retrieved, relevant, k),
        'Recall@K': recall_at_k(retrieved, relevant, k),
        'AveragePrecision@K': average_precision_at_k(retrieved, relevant, k),
        'F1Score@K': f1_score_at_k(retrieved, relevant, k),
        'NDCG@K': ndcg_at_k(retrieved, relevant, k)
    }

map_score = round(mean_average_precision(queries_retrieved, queries_relevant, k), 3)
mrr_score = round(mean_reciprocal_rank(queries_retrieved, queries_relevant), 3)

In [66]:
print(f"Query 1 (women full sleeve sweatshirt cotton):")
print(results[1])

print(f"\nQuery 2 (men slim jeans blue):")
print(results[2])

print(f"\nMAP: {map_score}")
print(f"MRR: {mrr_score}")


Query 1 (women full sleeve sweatshirt cotton):
{'Precision@K': 0.05, 'Recall@K': 0.07692307692307693, 'AveragePrecision@K': 0.004807692307692308, 'F1Score@K': 0.060606060606060615, 'NDCG@K': 0.24465054211822604}

Query 2 (men slim jeans blue):
{'Precision@K': 0.05, 'Recall@K': 0.1, 'AveragePrecision@K': 0.02, 'F1Score@K': 0.06666666666666667, 'NDCG@K': 0.38685280723454163}

MAP: 0.012
MRR: 0.131


### 2.3 Relevance Judgments and Analysis

You will act as expert judges by establishing the ground truth for each document and query.

a. For the test queries you defined in Part 1, Step 2 during indexing, assign a binary relevance label to each document: 1 if the document is relevant to the query, or 0 if it is not.

b. Comment on each of the evaluation metrics, stating how they differ, and which information gives each of them. Analyze your results.

c. Analyze the current search system and identify its main problems or limitations. For each issue you find, propose possible ways to resolve it. Consider aspects such as retrieval accuracy, ranking quality, handling of different field types, query formulation, and indexing strategies.

In [38]:
# TODO: ...